In [16]:
import torch
from torchsummary import summary

import torchvision
import torchvision.transforms as transforms

from resnet import ResNet, BasicBlock
from pyramidnet import PyramidNet
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpu_number = torch.cuda.device_count()

In [3]:
# load test set
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified


In [25]:
# load best performing models
# PyramidNet trained with SGD-M on 128 batch size with ABC augmentation
model = torch.load('./models/--batch_size 128 --net_type pyramidnet --optimizer sgd --augmentation ABC.pth')

# Initialize pyramidnet and load state dict that we trained
pyramidnet = PyramidNet('cifar10', 110, 84, 10).to(device)
pyramidnet.load_state_dict(model['net'])

In [27]:
summary(pyramidnet,(3, 32, 32)) # print torch summary to see architecture and number of parameters

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 18, 32, 32]           2,592
       BatchNorm2d-5           [-1, 18, 32, 32]              36
              ReLU-6           [-1, 18, 32, 32]               0
            Conv2d-7           [-1, 18, 32, 32]           2,916
       BatchNorm2d-8           [-1, 18, 32, 32]              36
        BasicBlock-9           [-1, 18, 32, 32]               0
      BatchNorm2d-10           [-1, 18, 32, 32]              36
           Conv2d-11           [-1, 19, 32, 32]           3,078
      BatchNorm2d-12           [-1, 19, 32, 32]              38
             ReLU-13           [-1, 19, 32, 32]               0
           Conv2d-14           [-1, 19,

In [32]:
# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = pyramidnet(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
acc = 100.*correct/total
print('Accuracy:', acc,'%')

Accuracy: 97.14 %


In [62]:
for test_images, test_labels in testloader:  # Sample two images to predict their labels
    sample_images = test_images[:2].to(device)    
    sample_labels = test_labels[:2]

outputs = pyramidnet(sample_images)
predictions = torch.argmax(outputs, axis=1)
print(f"Predicted labels: {predictions.cpu()} \nTrue labels: {sample_labels}")

Predicted labels: tensor([8, 3]) 
True labels: tensor([8, 3])
